In [2]:
import tensorflow as tf
print("GPU",tf.config.list_physical_devices("GPU"))
print(tf.test.is_gpu_available())
print(1)

GPU [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
1
